In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend

import numpy as np

from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse
import cv2
import os

# Define discriminator and generator

In [2]:
def custom_activation(output):
	logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
	result = logexpsum / (logexpsum + 1.0)
	return result

def define_discriminator(in_shape=(28,28,1), n_classes=10):
	in_image = Input(shape=in_shape)
 
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
	fe = LeakyReLU(alpha=0.2)(fe)

	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)

	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)

	fe = Flatten()(fe)
	fe = Dropout(0.4)(fe)
	fe = Dense(n_classes)(fe)

	c_out_layer = Activation('softmax')(fe)
	c_model = Model(in_image, c_out_layer)
	c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

	d_out_layer = Lambda(custom_activation)(fe)
	d_model = Model(in_image, d_out_layer)
	d_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

	return d_model, c_model

In [3]:
def define_generator(latent_dim):

	in_lat = Input(shape=(latent_dim,))

	n_nodes = 128 * 7 * 7

	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 128))(gen)

	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)

	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)

	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)

	model = Model(in_lat, out_layer)

	return model

In [4]:
def define_gan(g_model, d_model):

	d_model.trainable = False

	gan_output = d_model(g_model.output)

	model = Model(g_model.input, gan_output)

	opt = Adam(learning_rate=0.0002, beta_1=0.5)

	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# Tranining Semi-supervised GAN

In [5]:
from tensorflow.keras.datasets.mnist import load_data

def load_real_samples():

	(trainX, trainy), (_, _) = load_data()

	X = np.expand_dims(trainX, axis=-1)

	X = X.astype('float32')

	X = (X - 127.5) / 127.5

	print(X.shape, trainy.shape)
	return [X, trainy]

In [6]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
	
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
	
    for i in range(n_classes):

        X_with_class = X[y == i]

        ix = np.random.randint(0, len(X_with_class), n_per_class)

        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]

    return np.asarray(X_list), np.asarray(y_list)

In [7]:
# Function select real samples from dataset
def generate_real_samples(dataset, n_samples):

	images, labels = dataset

	ix = np.random.randint(0, images.shape[0], n_samples)

	X, labels = images[ix], labels[ix]
	y = np.ones((n_samples, 1))

	return [X, labels], y

In [8]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	
    gen_input = np.random.randn(latent_dim * n_samples)
    gen_input = gen_input.reshape(n_samples, latent_dim)

    images = generator.predict(gen_input)

    y = np.zeros((n_samples, 1))
    return images, y

In [9]:
OUTPUT_PATH = "output"

! mkdir "output"

mkdir: cannot create directory ‘output’: File exists


In [12]:
# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, batch_size=100):

    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)

    bat_per_epo = int(dataset[0].shape[0] / batch_size)
    n_steps = bat_per_epo * n_epochs
    half_batch = int(batch_size / 2)
    print('n_epochs = %d, batch_size = %d, 1/2 = %d, b/e = %d, steps = %d' % (n_epochs, batch_size, half_batch, bat_per_epo, n_steps))

    for i in range(n_steps):

        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)

        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)

        # update generator (g)
        X_gan = np.random.randn(latent_dim * batch_size)
        X_gan = X_gan.reshape(batch_size, latent_dim)
        y_gan = np.ones((batch_size, 1))

        g_loss = gan_model.train_on_batch(X_gan, y_gan)

        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:

            epoch = (i + 1) / bat_per_epo

            # summarize loss on this epoch
            print('[INFO] %d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (epoch, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
            
            p = [OUTPUT_PATH, "epoch_{}_output.png".format(str(epoch))]

            images, _ = generate_fake_samples(g_model, latent_dim, 64)

            images = ((images * 127.5) + 127.5).astype("uint8")
            images = np.repeat(images, 3, axis=-1)
            vis = build_montages(images, (28, 28), (8, 8))[0]

            # write the visualization to disk
            p = os.path.sep.join(p)
            print("path: ", p)
            cv2.imwrite(p, vis)

            # evaluate the classifier model
            X, y = dataset
            _, acc = c_model.evaluate(X, y, verbose=0)
            print('Classifier Accuracy: %.3f%%' % (acc * 100))

            # save the generator model
            filename2 = 'g_model_%04d.h5' % (i+1)
            g_model.save(filename2)
            
            # save the classifier model
            filename3 = 'c_model_%04d.h5' % (i+1)
            c_model.save(filename3)
            print('>Saved: %s, and %s' % (filename2, filename3))

In [13]:
latent_dim = 100

d_model, c_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

(60000, 28, 28, 1) (60000,)
(100, 28, 28, 1) (100,)
n_epochs = 20, batch_size = 100, 1/2 = 50, b/e = 600, steps = 12000
[INFO] 1, c[0.032,100], d[0.606,0.577], g[1.549]
path:  output/epoch_01.0_output.png
Classifier Accuracy: 83.025%
>Saved: g_model_0600.h5, and c_model_0600.h5
[INFO] 2, c[0.013,100], d[0.563,0.663], g[1.482]
path:  output/epoch_02.0_output.png
Classifier Accuracy: 89.332%
>Saved: g_model_1200.h5, and c_model_1200.h5
[INFO] 3, c[0.019,100], d[0.652,0.603], g[1.500]
path:  output/epoch_03.0_output.png
Classifier Accuracy: 91.265%
>Saved: g_model_1800.h5, and c_model_1800.h5
[INFO] 4, c[0.006,100], d[0.741,0.630], g[1.126]
path:  output/epoch_04.0_output.png
Classifier Accuracy: 93.453%
>Saved: g_model_2400.h5, and c_model_2400.h5
[INFO] 5, c[0.005,100], d[0.720,0.825], g[1.324]
path:  output/epoch_05.0_output.png
Classifier Accuracy: 93.792%
>Saved: g_model_3000.h5, and c_model_3000.h5
[INFO] 6, c[0.004,100], d[0.701,0.674], g[1.281]
path:  output/epoch_06.0_output.png
